In [1]:
from sklearn.ensemble import RandomForestClassifier,StackingClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV

In [2]:
df = pd.read_csv('train.csv')
df.drop(columns=["Ticket",'Cabin','PassengerId',"Name"],inplace=True)
df = pd.get_dummies(df,columns=['Embarked',"Sex"])


In [4]:
X = df.drop(columns = ['Survived'])
y = df['Survived']
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=42)

In [7]:

base_estimators = [
    ("rf",RandomForestClassifier()),
    ('dt',DecisionTreeClassifier(random_state=41))
]
final_estimator = GradientBoostingClassifier()

model = StackingClassifier(
    estimators = base_estimators,
    final_estimator = final_estimator
)
param_grid = {
    'dt__max_depth':[2,4,6],
    'final_estimator__n_estimators':[70,80,90,100]
}
grid = GridSearchCV(model,param_grid=param_grid,scoring='accuracy',cv =5 ,n_jobs=-1)
grid.fit(X,y)

InvalidParameterError: The 'scoring' parameter of GridSearchCV must be a str among {'top_k_accuracy', 'neg_mean_poisson_deviance', 'precision_micro', 'neg_median_absolute_error', 'recall_macro', 'f1', 'average_precision', 'recall_weighted', 'f1_macro', 'jaccard_samples', 'adjusted_mutual_info_score', 'jaccard_micro', 'f1_weighted', 'explained_variance', 'neg_mean_absolute_error', 'normalized_mutual_info_score', 'neg_mean_absolute_percentage_error', 'recall_samples', 'neg_negative_likelihood_ratio', 'recall_micro', 'precision_macro', 'roc_auc_ovo', 'neg_brier_score', 'homogeneity_score', 'neg_root_mean_squared_log_error', 'neg_mean_gamma_deviance', 'jaccard', 'jaccard_weighted', 'adjusted_rand_score', 'balanced_accuracy', 'precision_samples', 'jaccard_macro', 'roc_auc', 'roc_auc_ovo_weighted', 'precision', 'roc_auc_ovr', 'r2', 'positive_likelihood_ratio', 'matthews_corrcoef', 'neg_root_mean_squared_error', 'neg_mean_squared_log_error', 'f1_samples', 'neg_max_error', 'd2_absolute_error_s

In [ ]:

print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)